In [ ]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='123myprojectid', project_access_token='123myaccesstoken')
pc = project.project_context

In [ ]:
wa_version = "2020-09-24"
wa_apikey = "123myapikey"
wa_url = "https://gateway.watsonplatform.net/assistant/api/"
wa_skill = "skillid"

In [ ]:
import json
!pip install ibm-watson
import os
import pandas as panda
import ibm_watson
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

authenticator = IAMAuthenticator(wa_apikey)

assistant_service=ibm_watson.AssistantV1(
    version = wa_version,
    authenticator = authenticator
)

assistant_service.set_service_url(wa_url)

In [ ]:
workspace_response = assistant_service.get_workspace(
    workspace_id = wa_skill,
    export=True
).get_result()

formattedOutputDict = []

for node in workspace_response['dialog_nodes']:
    outputText = ""
    if 'output' in node:
        if 'generic' in node['output']:
            for item in node['output']['generic']:
                if item['response_type'] == "text":
                    for response in item['values']:
                        outputText += " " + response['text'] + " "
                if item['response_type'] == "option":
                    outputText += str(item)
                if item['response_type'] == "connect_to_agent":
                    outputText += " CONNECT TO AGENT "
                if item['response_type'] == "search_skill":
                    outputText += " SEARCH SKILL "

                if 'title' in node:
                    title = node['title']
                else:
                    title = 'NONE'
                if 'conditions' in node:
                    condition = node['conditions']
                else:
                    condition = 'NONE'
                    
    if len(outputText) > 0:
        thisNode = {'title': title, 'conditions': condition, 'outputText': outputText}
        formattedOutputDict.append(thisNode)

df = panda.DataFrame(formattedOutputDict)
# print(df)
project.save_data("dialog_output.csv", df.to_csv(index=False), overwrite=True)

print("Dialog downloaded ✅")